In [ ]:
using MeshCat
using RigidBodyDynamics
using MeshCatMechanisms

In [ ]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
IJuliaCell(vis) # render the visualizer here inside the jupyter notebook

In [ ]:
urdf = joinpath(Pkg.dir("MeshCatMechanisms"), "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
delete!(vis)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
set_configuration!(mvis, [0.0, 0.0])

In [ ]:
state = MechanismState(robot, randn(2), randn(2))
t, q, v = simulate(state, 5.0);

animate(mvis, t, q)

### Visualizing Frames and Points

You can use the lower-level `setelement!()` function to attach geometries to frames in the mechanism. For example, to visualize a coordinate frame with a triad, you can do: 

In [ ]:
lower_arm = bodies(robot)[end]
body_frame = default_frame(lower_arm)
setelement!(mvis, body_frame)

And likewise, to visualize a point attached to a frame, you can do:

In [ ]:
radius = 0.05
name = "my_point"
setelement!(mvis, Point3D(body_frame, 0.2, 0.2, 0.2), radius, name)

The triad and point are included in the MeshCat scene tree, so each will move along with the body to which it's attached:

In [ ]:
animate(mvis, t, q)

In [ ]:
using ValkyrieRobot

val = Valkyrie();
delete!(vis)
mvis = MechanismVisualizer(
    val.mechanism, 
    URDFVisuals(ValkyrieRobot.urdfpath(), package_path=[dirname(dirname(ValkyrieRobot.urdfpath()))]),
    vis);

In [ ]:
using ValkyrieRobot.BipedControlUtil: Side, flipsign_if_right

function initialize!(state::MechanismState, val::Valkyrie)
    zero!(state)
    mechanism = val.mechanism
    for side in instances(Side)
        set_configuration!(state, findjoint(mechanism, "$(side)KneePitch"), [1.205])
        set_configuration!(state, findjoint(mechanism, "$(side)HipPitch"), [-0.49])
        set_configuration!(state, findjoint(mechanism, "$(side)AnklePitch"), [-0.71])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderPitch"), [0.300196631343025])
        set_configuration!(state, findjoint(mechanism, "$(side)ShoulderRoll"), [flipsign_if_right(-1.25, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ElbowPitch"), [flipsign_if_right(-0.785398163397448, side)])
        set_configuration!(state, findjoint(mechanism, "$(side)ForearmYaw"), [1.571])
    end
    set_configuration!(state, val.basejoint, [1; 0; 0; 0; 0; 0; 1.025])
    nothing
end
state = MechanismState(val.mechanism)
initialize!(state, val)
set_configuration!(mvis, configuration(state))

In [ ]:
# Handling robots with fixed joints
urdf = joinpath(Pkg.dir("MeshCatMechanisms"), "test", "urdf", "Acrobot.urdf")
robot = parse_urdf(Float64, urdf)
RigidBodyDynamics.remove_fixed_tree_joints!(robot)
IJuliaCell(MechanismVisualizer(robot, URDFVisuals(urdf)))